In [1]:
! wget https://vectorbase.org/common/downloads/release-55/AgambiaePEST/gff/data/VectorBase-55_AgambiaePEST.gff -O gambiae.gff

--2022-10-21 20:45:32--  https://vectorbase.org/common/downloads/release-55/AgambiaePEST/gff/data/VectorBase-55_AgambiaePEST.gff
Resolving vectorbase.org (vectorbase.org)... 128.91.204.54
Connecting to vectorbase.org (vectorbase.org)|128.91.204.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23512903 (22M) [application/x-gff]
Saving to: ‘gambiae.gff’

gambiae.gff         100%[===================>]  22.42M  30.6MB/s    in 0.7s    

2022-10-21 20:45:33 (30.6 MB/s) - ‘gambiae.gff’ saved [23512903/23512903]



In [2]:
! gzip -9 gambiae.gff

Let’s start by creating an annotation database with gffutils, based on our GFF file:

In [4]:
import gffutils
import sqlite3

# The gffutils library creates a SQLite database to store annotations efficiently. 
# Here, we will try to create the database, but if it already exists, we will use 
# the existing one. This step can be time-consuming.

try:
    db = gffutils.create_db('gambiae.gff.gz', 'ag.db')
except sqlite3.OperationalError:
    db = gffutils.FeatureDB('ag.db')

Now, let’s list all the available feature types and count them:

These features will include contigs, genes, exons, transcripts, and so on. Note that we will use
the gffutils package’s featuretypes function.

In [5]:
print(list(db.featuretypes()))
for feat_type in db.featuretypes():
    print(feat_type, db.count_features_of_type(feat_type))

['CDS', 'RNase_MRP_RNA', 'RNase_P_RNA', 'SRP_RNA', 'exon', 'five_prime_UTR', 'lnc_RNA', 'mRNA', 'ncRNA', 'ncRNA_gene', 'pre_miRNA', 'protein_coding_gene', 'pseudogene', 'pseudogenic_transcript', 'rRNA', 'snRNA', 'snoRNA', 'tRNA', 'three_prime_UTR']
CDS 67394
RNase_MRP_RNA 1
RNase_P_RNA 1
SRP_RNA 3
exon 61590
five_prime_UTR 17472
lnc_RNA 2
mRNA 15125
ncRNA 4
ncRNA_gene 729
pre_miRNA 77
protein_coding_gene 13094
pseudogene 9
pseudogenic_transcript 9
rRNA 242
snRNA 35
snoRNA 2
tRNA 362
three_prime_UTR 12236


Let’s list all seqids:

This will show us that there is annotation information for all chromosome arms and sex
chromosomes, mitochondrion, and the unknown chromosome.

In [6]:
seqids = set()
for e in db.all_features():
    seqids.add(e.seqid)
for seqid in seqids:
    print(seqid)

AgamP4_3R
AgamP4_Mt
AgamP4_2R
AgamP4_2L
AgamP4_Y_unplaced
AgamP4_X
AgamP4_3L
AgamP4_UNKN


Now, let’s extract a lot of useful information per chromosome, such as the number of genes, number of transcripts per gene, number of exons, and so on:

We will traverse all seqids while extracting all protein-coding genes (using region). In each
gene, we count the number of alternative transcripts. If there are none (note that this is probably
an annotation issue and not a biological one), we count the exons (children). If there are
several transcripts, we count the exons per transcript. We also account for the span size to
check for the gene that spans the largest region. 

We follow a similar procedure to find the gene and the largest number of exons. Finally, we print a
dictionary that contains the distribution of the number of alternative transcripts per gene (num_mRNAs)
and the distribution of the number of exons per transcript (num_exons).

In [8]:
from collections import defaultdict

num_mRNAs = defaultdict(int)
num_exons = defaultdict(int)
max_exons = 0
max_span = 0
for seqid in seqids:
    cnt = 0
    for gene in db.region(seqid=seqid, featuretype='protein_coding_gene'):
        cnt += 1
        span = abs(gene.start - gene.end) # strand
        if span > max_span:
            max_span = span
            max_span_gene = gene
        my_mRNAs = list(db.children(gene, featuretype='mRNA'))
        num_mRNAs[len(my_mRNAs)] += 1
        if len(my_mRNAs) == 0:
            exon_check = [gene]
        else:
            exon_check = my_mRNAs
        for check in exon_check:
            my_exons = list(db.children(check, featuretype='exon'))
            num_exons[len(my_exons)] += 1
            if len(my_exons) > max_exons:
                max_exons = len(my_exons)
                max_exons_gene = gene
    print(f'seqid {seqid}, number of genes {cnt}')
print('Max number of exons: %s (%d)' % (max_exons_gene.id, max_exons))
print('Max span: %s (%d)' % (max_span_gene.id, max_span))
print(num_mRNAs)
print(num_exons)

seqid AgamP4_3R, number of genes 2692
seqid AgamP4_Mt, number of genes 13
seqid AgamP4_2R, number of genes 3670
seqid AgamP4_2L, number of genes 2950
seqid AgamP4_Y_unplaced, number of genes 2
seqid AgamP4_X, number of genes 1073
seqid AgamP4_3L, number of genes 2216
seqid AgamP4_UNKN, number of genes 478
Max number of exons: AGAP001660 (69)
Max span: AGAP006656 (365621)
defaultdict(<class 'int'>, {1: 11753, 3: 234, 2: 964, 7: 5, 9: 3, 4: 85, 8: 4, 6: 10, 13: 2, 5: 27, 11: 3, 20: 1, 10: 1, 12: 2})
defaultdict(<class 'int'>, {1: 1207, 3: 2763, 9: 459, 4: 2059, 6: 1126, 7: 786, 2: 3159, 5: 1552, 13: 119, 11: 282, 8: 530, 18: 46, 17: 76, 15: 65, 10: 340, 12: 194, 21: 27, 24: 9, 16: 65, 14: 96, 19: 32, 20: 36, 28: 6, 22: 10, 23: 15, 30: 5, 27: 6, 37: 1, 25: 15, 26: 8, 29: 6, 32: 7, 35: 1, 33: 3, 45: 1, 38: 1, 46: 1, 69: 1, 31: 10})
